In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(api_wrapper=TavilySearchAPIWrapper())


from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())



available_tools = [tavily_tool, wikipedia]






In [10]:
properties_dataset_path = "datasets/apify_rightmove_london_properties.pkl"

In [171]:
import pickle as pkl

properties_dataset = pkl.load(open(properties_dataset_path,'rb'))


irrelevant_keys = "descriptionHtml archived tags epc published sold floorplans brochures agentDescriptionHtml agentListingsUrl agentProfileUrl images yearsRemainingOnLease secondaryPrice coordinates councilTaxIncluded annualServiceCharge councilTaxBand domesticRates groundRentPercentageIncrease groundRentReviewPeriodInYears annualGroundRent councilTaxExempt agentDisplayAddress agentLogo agentPhone agent deliveryPointId countryCode deliveryPointId ukCountry outcode incode".split(' ')
for prop in properties_dataset:
    for r in irrelevant_keys:
        if r in prop:
            del prop[r]
    prop['price'] = int(prop['price'][1:].replace(',',''))
    prop['propertyType'] = prop['propertyType'].lower()


keys = []
for prop in properties_dataset:
    keys+=prop.keys()
keys = list(set(keys))
keys

['tenure',
 'type',
 'url',
 'propertyType',
 'nearestStations',
 'sizeSqFeetMin',
 'sizeSqFeetMax',
 'displayAddress',
 'bathrooms',
 'bedrooms',
 'description',
 'title',
 'addedOn',
 'features',
 'id',
 'price']

In [181]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
class NearestStation(BaseModel):
    name: str = Field()
    distance: float = Field(description="Distance in miles")

class PropertyInfo(BaseModel):
    tenture: str = Field(description="Tenture Type/Duration")
    type: str = Field(description="Sales or Rental")
    url: str = Field(description="Url of property")
    nearestStations: List[NearestStation] = Field(description="Nearest available stations")
    bathrooms: int = Field(description="Number of bathrooms")
    bedrooms: int = Field(description="Number of Bedrooms")
    description: int = Field(description="Description of property")
    features: List[str] = Field(description="Features of Property")
    price: int = Field("Price of property in euro")


class OutputFormat(BaseModel):
    executable_fn: str = Field(description="lambda function in python to run in repl and store the function in a string"

desc_dem = properties_dataset[0]
desc_dem

{'id': '142894421',
 'url': 'https://www.rightmove.co.uk/properties/142894421#/?channel=RES_BUY',
 'title': '6 bedroom house for sale in Princes Gate, London, SW7',
 'displayAddress': 'Princes Gate, London, SW7',
 'bathrooms': 7,
 'bedrooms': 6,
 'propertyType': 'house',
 'price': 29500000,
 'type': 'sale',
 'description': "United Kingdom Sotheby's International Realty is proud to present this exceptional Grade II-listed house that spans an impressive 9,430 sq. ft in South Kensington. With its beautiful stucco façade, the exterior of the building echoes the grandeur beyond. Unfolding over five floors, the house includes a wonderfully planned lower-ground floor and a contemporary loft space.\n\nEntry is to the ground floor, with a large dual-aspect reception room on the left; this room can be separated into a living space and study by neat bi-fold doors as required. At the back of the plan is another sitting room, which overlooks the terrace and has beautifully bespoke cabinetry.\n\nOn 

In [177]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [186]:
from langchain_core.prompts import PromptTemplate


from langchain.output_parsers import PydanticOutputParser



parser = PydanticOutputParser(pydantic_object=PropertyInfo)
schema = parser.get_format_instructions().split("Here is the output schema:")[1].strip()[3:][:-3].strip()

template = (
    "We've a list of real estate properties in london information in a list of dictionaries in python in the following schema/format"
    "{formatt}\n"
    "Based on user query, generate a lambda function which will be iterated through the list to filter out the best property match for user"
    "The user query is following: \n"
    "{query}\n"
    ""
)
prompt = PromptTemplate.from_template(template, partial_variables=dict(formatt=parser.get_format_instructions()))
chain = prompt | llm 
res = chain.invoke(dict(query="I need a house in london with minimum of 2 bedrooms and price above 20k euros. make sure the nearest station is at least half a mile near")).content
print(res)

```python
lambda property: property["type"] == "House" and property["bedrooms"] >= 2 and property["price"] > 20000 and any(station["distance"] <= 0.5 for station in property["nearestStations"])
```


In [191]:
parser.get_format_instructions()

'{"title": "PydanticOutputParser", "description": "Parse an output using a pydantic model.", "type": "object", "properties": {"name": {"title": "Name", "type": "string"}, "diff": {"title": "Diff", "default": false, "type": "boolean"}, "pydantic_object": {"title": "Pydantic Object"}}, "required": ["pydantic_object"]}'

'{"properties": {"tenture": {"title": "Tenture", "description": "Tenture Type/Duration", "type": "string"}, "type": {"title": "Type", "description": "Sales or Rental", "type": "string"}, "url": {"title": "Url", "description": "Url of property", "type": "string"}, "nearestStations": {"title": "Neareststations", "description": "Nearest available stations", "type": "array", "items": {"$ref": "#/definitions/NearestStation"}}, "bathrooms": {"title": "Bathrooms", "description": "Number of bathrooms", "type": "integer"}, "bedrooms": {"title": "Bedrooms", "description": "Number of Bedrooms", "type": "integer"}, "description": {"title": "Description", "description": "Description of property", "type": "integer"}, "features": {"title": "Features", "description": "Features of Property", "type": "array", "items": {"type": "string"}}, "price": {"title": "Price", "default": "Price of property in euro", "type": "integer"}}, "required": ["tenture", "type", "url", "nearestStations", "bathrooms", "bedroo